# Text Data Cleansing For Sentiment Analysis

Before we create models for sentiment analysis, we need data for the basis of this analysis. This data contains large number of text data with its sentiment that has been identified before. To create a better model, we might need to preprocess this text data into a form that would work better in modelling.

## Preparing The Environment

In [2]:
# Import library
import pandas as pd
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [3]:
# Import text data
df = pd.read_csv(
    "csv_data/train_preprocess.txt",
    sep="\t",
    names=["kalimat", "sentiment"]
)
df

,kalimat,sentiment
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative
...,...,...
10995,tidak kecewa,positive
10996,enak rasa masakan nya apalagi kepiting yang me...,positive
10997,hormati partai-partai yang telah berkoalisi,neutral
10998,"pagi pagi di tol pasteur sudah macet parah , b...",negative


In [4]:
# import additional data
# kamus abusive
abusive_data = pd.read_csv("csv_data/abusive.csv")

# kamus alay
kamus_alay_data = pd.read_csv("csv_data/alay.csv", encoding="latin-1")
kamus_alay_data.columns = ['alay', 'baku']
kamus_alay_dict = dict(zip(kamus_alay_data['alay'], kamus_alay_data['baku']))

# stop word indonesia
stopword_id = pd.read_csv("csv_data/stopword.csv")

## Process Text Data

### Define function for string cleaning

In [9]:
def text_cleansing(text):
    # lowercase
    clean_text = str(text).lower()
    # clean URL
    clean_text = re.sub(r'(http\S+|www\S+)', '', clean_text)
    # clean punctuations
    clean_text = re.sub(r'[^a-zA-Z0-9\s]', ' ', clean_text)
    # clean multiple whitespace
    clean_text = ' '.join(clean_text.split())
    return clean_text

In [6]:
def abusive_censor(text):
    for word in abusive_data['ABUSIVE']:
        if word == text:
            text = text.replace(word, '*' * 3)
    return text
def abusive_cleansing(text):
    clean_text = ' '.join([abusive_censor(i) for i in text.split()])
    return clean_text

In [7]:
def alay_replace(text):
    for key, value in kamus_alay_dict.items():
        if key == text:
            text = text.replace(key, value)
    return text
def alay_cleansing(text):
    clean_text = ' '.join([alay_replace(j) for j in text.split()])
    return clean_text

In [10]:
df['clean text'] = df['kalimat'].apply(text_cleansing)
df

,kalimat,sentiment,clean text
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive,warung ini dimiliki oleh pengusaha pabrik tahu...
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral,mohon ulama lurus dan k212 mmbri hujjah partai...
2,lokasi strategis di jalan sumatera bandung . t...,positive,lokasi strategis di jalan sumatera bandung tem...
3,betapa bahagia nya diri ini saat unboxing pake...,positive,betapa bahagia nya diri ini saat unboxing pake...
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative,duh jadi mahasiswa jangan sombong dong kasih k...
...,...,...,...
10995,tidak kecewa,positive,tidak kecewa
10996,enak rasa masakan nya apalagi kepiting yang me...,positive,enak rasa masakan nya apalagi kepiting yang me...
10997,hormati partai-partai yang telah berkoalisi,neutral,hormati partai partai yang telah berkoalisi
10998,"pagi pagi di tol pasteur sudah macet parah , b...",negative,pagi pagi di tol pasteur sudah macet parah bik...


### Removes stopword from text

In [11]:
# duplicate dataset
df_2 = df.copy()

In [12]:
# Define function for removing stopword
def remove_stopwords(text):
    stopword = stopword_id['stopword'].values
    clean_text = [word for word in text.split() if word.lower() not in stopword]
    clean_text = ' '.join(clean_text)
    return clean_text

In [13]:
# Removes stopword
df_2['clean stopword'] = df['clean text'].apply(remove_stopwords)
df_2

,kalimat,sentiment,clean text,clean stopword
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive,warung ini dimiliki oleh pengusaha pabrik tahu...,warung dimiliki pengusaha pabrik puluhan terke...
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral,mohon ulama lurus dan k212 mmbri hujjah partai...,mohon ulama lurus k212 mmbri hujjah partai diw...
2,lokasi strategis di jalan sumatera bandung . t...,positive,lokasi strategis di jalan sumatera bandung tem...,lokasi strategis jalan sumatera bandung nyaman...
3,betapa bahagia nya diri ini saat unboxing pake...,positive,betapa bahagia nya diri ini saat unboxing pake...,betapa bahagia unboxing paket barang bagus men...
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative,duh jadi mahasiswa jangan sombong dong kasih k...,duh mahasiswa sombong kasih kartu kuning belaj...
...,...,...,...,...
10995,tidak kecewa,positive,tidak kecewa,tidak kecewa
10996,enak rasa masakan nya apalagi kepiting yang me...,positive,enak rasa masakan nya apalagi kepiting yang me...,enak masakan kepiting menyenangkan memilih kep...
10997,hormati partai-partai yang telah berkoalisi,neutral,hormati partai partai yang telah berkoalisi,hormati partai partai berkoalisi
10998,"pagi pagi di tol pasteur sudah macet parah , b...",negative,pagi pagi di tol pasteur sudah macet parah bik...,pagi pagi tol pasteur macet parah bikin jengkel


## Export to csv

In [14]:
df_2.to_csv(
    "csv_data/cleaned_text.csv", 
    sep='\t'
)